In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/10 17:42:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
players_schema = StructType([
    StructField("player_id", IntegerType(), True),
    StructField("group_id", IntegerType(), True)
])

players_data = [
    (15, 1),
    (25, 1),
    (30, 1),
    (45, 1),
    (10, 2),
    (35, 2),
    (50, 2),
    (20, 3),
    (40, 3)
]

players_df = spark.createDataFrame(players_data, schema=players_schema)

matches_schema = StructType([
    StructField("match_id", IntegerType(), True),
    StructField("first_player", IntegerType(), True),
    StructField("second_player", IntegerType(), True),
    StructField("first_score", IntegerType(), True),
    StructField("second_score", IntegerType(), True)
])

matches_data = [
    (1, 15, 45, 3, 0),
    (2, 30, 25, 1, 2),
    (3, 30, 15, 2, 0),
    (4, 40, 20, 5, 2),
    (5, 35, 50, 1, 1)
]

matches_df = spark.createDataFrame(matches_data, schema=matches_schema)

players_df.show()
matches_df.show()

print(matches_df.dtypes)

+---------+--------+
|player_id|group_id|
+---------+--------+
|       15|       1|
|       25|       1|
|       30|       1|
|       45|       1|
|       10|       2|
|       35|       2|
|       50|       2|
|       20|       3|
|       40|       3|
+---------+--------+

+--------+------------+-------------+-----------+------------+
|match_id|first_player|second_player|first_score|second_score|
+--------+------------+-------------+-----------+------------+
|       1|          15|           45|          3|           0|
|       2|          30|           25|          1|           2|
|       3|          30|           15|          2|           0|
|       4|          40|           20|          5|           2|
|       5|          35|           50|          1|           1|
+--------+------------+-------------+-----------+------------+

[('match_id', 'int'), ('first_player', 'int'), ('second_player', 'int'), ('first_score', 'int'), ('second_score', 'int')]


In [5]:
players_df.createOrReplaceTempView("Players")
matches_df.createOrReplaceTempView("Matches")

In [13]:
result = spark.sql(
    """
    select distinct group_id, 
        first_value(pscores.player_id) 
            over(partition by group_id order by score desc, pscores.player_id ASC) as player_id
    from (select player_id, sum(score) as score
    from (select first_player as player_id, first_score as score from Matches
    UNION ALL 
    select second_player as player_id, second_score as score from Matches) all
    group by 1) pscores
        left join Players p on pscores.player_id = p.player_id
    """
)
result.show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+



In [27]:
result = spark.sql(
    """          
        SELECT group_id, player_id FROM 
(
    SELECT
        group_id, player_id, rank() over(PARTITION BY group_id ORDER BY score desc, player_id) rk
    FROM (
        SELECT 
            p.group_id
            , p.player_id
            , sum(if(p.player_id=m.first_player,first_score,second_score)) score
        FROM Players p LEFT JOIN Matches m on p.player_id = m.first_player or p.player_id = m.second_player
        GROUP BY p.group_id, p.player_id
    ) a
) b where rk=1;
    
    """
)
result.show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+

